## 1. 뉴스제목 가져오기
* user-agent 요청헤더를 반드시 설정해야 한다.

In [25]:
# requests 라이브러리 설치여부 확인

In [26]:
# beautifulsoup4 라이브러리 설치여부 확인

In [27]:
# reqeusts, bs4 import

# BeautifulSoup 클래스 import


In [28]:
# requests, bs4 버전 확인하기

### 1. 뉴스 제목 추출하기

In [1]:
# IT/과학 뉴스 
req_param = {
    'sid': 105
}

sid = 104
#url = f'https://news.naver.com/section/{sid}' 
url = 'https://news.naver.com/section/{sid}'.format(**req_param)
print(url)

# requests 의 get() 함수 호출하기 

# 응답(response)이 OK 이면
# 응답 (response)에서 text 추출
# BeautifulSoup 객체 생성  
# CSS 선택자
# print(soup.select("div.sa_text a[href*='mnews/article']"))

# <a> 태그 리스트 순회하기    

# 응답(response)이 Error 이면 status code 출력    



https://news.naver.com/section/105


### 1.1 뉴스제목 추출하는 함수 선언하기

In [3]:
import requests
from bs4 import BeautifulSoup

#section_dict = {100:'정치',101:'경제',102:'사회',103:'생활/문화',104:'세계',105:'IT/과학'}
section_dict = {'정치':100,'경제':101,'사회':102,'생활/문화':103,'세계':104,'IT/과학':105}

def print_news(section):  #print_new(103,'생활/문화') 
    sid = section_dict[section]
    url = f'https://news.naver.com/section/{sid}'
    print(f'{url} {section} 뉴스')

    # 요청 헤더 설정 : 브라우저 정보
    req_header = {
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36"
    }
    # requests 의 get() 함수 호출하기 
    res=requests.get(url,headers=req_header)
    print(res.status_code, res.ok)
    if res.ok:
        html = res.text
        # BeautifulSoup 객체 생성
        soup = BeautifulSoup(html, "html.parser")
        a_tags = soup.select("div.sa_text a[href*='https://n.news.naver.com/mnews/article'].sa_text_title")
        print(len(a_tags))
        # <a> 태그 리스트 순회하기
        for idx,a_tag in enumerate(a_tags,1):        
            if a_tag.text:
                title = a_tag.text.strip() # 제목
                link = a_tag['href'] # 링크 <a href="">
                #print(len(title))
                print(f'{idx} {title} {link}')
    else:
        # 응답(response)이 Error 이면 status code 출력    
        print(f'Error Code = {res.status_code}')       
    
    

In [ ]:
print_news(103,section_dict[103])

### 2. Image 다운로드
* referer 요청 헤더를 반드시 설정해야 한다.

In [5]:
import requests
import os

req_header = {
    'referer':'https://comic.naver.com/webtoon/detail?titleId=835751&no=24&week=tue',
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36"
}

img_urls = [
    'https://image-comic.pstatic.net/webtoon/835751/24/20250623151113_4a032791c141c6f022bdc38c218efdf2_IMAG01_1.jpg',
    'https://image-comic.pstatic.net/webtoon/835751/24/20250623151113_4a032791c141c6f022bdc38c218efdf2_IMAG01_2.jpg',
    'https://image-comic.pstatic.net/webtoon/835751/24/20250623151113_4a032791c141c6f022bdc38c218efdf2_IMAG01_3.jpg'
]

for img_url in img_urls:
    # requests 의 get(url, headers) 함수 호출하기 
    res = requests.get(img_url,headers=req_header)
    print(res.ok, res.status_code)
    if res.ok:      
        # binary 응답 데이터 가져오기
        img_data = res.content
        # url에서 파일명만 추출하기
        file_name = os.path.basename(img_url)
        # binday data를 file에 write하기
        with open(file_name, 'wb') as file:
            print(f'Writing to {file_name}({len(img_data):,} bytes)')
            file.write(img_data)

True 200
Writing to 20250623151113_4a032791c141c6f022bdc38c218efdf2_IMAG01_1.jpg(110,240 bytes)
True 200
Writing to 20250623151113_4a032791c141c6f022bdc38c218efdf2_IMAG01_2.jpg(157,933 bytes)
True 200
Writing to 20250623151113_4a032791c141c6f022bdc38c218efdf2_IMAG01_3.jpg(143,514 bytes)


* 현재 요청된 페이지의 image 모두 다운로드 해보기

In [6]:
import requests
from bs4 import BeautifulSoup
import os

webtoon_url = 'https://comic.naver.com/webtoon/detail?titleId=835751&no=24&week=tue'
req_header = {
    'referer':webtoon_url,
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36"
}
res = requests.get(webtoon_url)
print(res.status_code)
if res.ok:
    soup = BeautifulSoup(res.text, 'html.parser')
    print(len(soup.select("img[src$='.jpg']")))
    #https://image-comic.pstatic.net/webtoon
    print(len(soup.select("img[src^='https://image-comic.pstatic.net/webtoon']")))
    print(len(soup.select("img[src*='IMAG01']"))) # 일치함
    # img src file명을 저장할 list 선언
    img_url_list = list() #['http://xxx/aa.jpg']
    img_tags = soup.select("img[src*='IMAG01']") #img_tags의 타입은 Resultset [Tag,Tag,]
    for img_tag in img_tags: #img_tag 의 타입은 Tag <img>
        img_url = img_tag['src']
        img_url_list.append(img_url)
    #print(len(img_url_list), img_url_list[0:2])

    # img 디렉토리가 없으면 생성하기
    imgdir_name = 'img'
    if not os.path.isdir(imgdir_name):
        os.mkdir(imgdir_name)

    for jpg_url in img_url_list:
        res = requests.get(jpg_url, headers=req_header)
        if res.ok:
            # binary data 추출하기
            img_data = res.content
            # img/xxx.jpg 디렉토리명과 jpg파일명을 join
            dir_name = os.path.join(imgdir_name,os.path.basename(jpg_url))
            with open(dir_name,'wb') as file:
                print(f'Writing to {dir_name}({len(img_data):,} bytes)')
                file.write(img_data)
        else:
           print(f'Error Code = {res.status_code}')             

200
88
88
62
Writing to img\20250623151113_4a032791c141c6f022bdc38c218efdf2_IMAG01_1.jpg(110,240 bytes)
Writing to img\20250623151113_4a032791c141c6f022bdc38c218efdf2_IMAG01_2.jpg(157,933 bytes)
Writing to img\20250623151113_4a032791c141c6f022bdc38c218efdf2_IMAG01_3.jpg(143,514 bytes)
Writing to img\20250623151113_4a032791c141c6f022bdc38c218efdf2_IMAG01_4.jpg(165,592 bytes)
Writing to img\20250623151113_4a032791c141c6f022bdc38c218efdf2_IMAG01_5.jpg(129,768 bytes)
Writing to img\20250623151113_4a032791c141c6f022bdc38c218efdf2_IMAG01_6.jpg(171,935 bytes)
Writing to img\20250623151113_4a032791c141c6f022bdc38c218efdf2_IMAG01_7.jpg(105,653 bytes)
Writing to img\20250623151113_4a032791c141c6f022bdc38c218efdf2_IMAG01_8.jpg(81,579 bytes)
Writing to img\20250623151113_4a032791c141c6f022bdc38c218efdf2_IMAG01_9.jpg(134,745 bytes)
Writing to img\20250623151113_4a032791c141c6f022bdc38c218efdf2_IMAG01_10.jpg(117,224 bytes)
Writing to img\20250623151113_4a032791c141c6f022bdc38c218efdf2_IMAG01_11.jpg(

리팩토링 코드

In [8]:
import requests
from bs4 import BeautifulSoup
import os

# 기본 설정
url = 'https://comic.naver.com/webtoon/detail?titleId=833255&no=35&week=tue'
req_header = {'referer': url}
imgdir_name = 'img'

# 이미지 저장 폴더가 없으면 생성
os.makedirs(imgdir_name, exist_ok=True)

# 웹 페이지 요청 및 확인
res = requests.get(url)
if not res.ok:
    print(f'Error Code = {res.status_code}')
    exit()

# 이미지 URL 추출
soup = BeautifulSoup(res.text, 'html.parser')
img_url_list = [img_tag['src'] for img_tag in soup.select("img[src*='IMAG01']")]

# 이미지 다운로드
for img_url in img_url_list:
    res = requests.get(img_url, headers=req_header)
    if res.ok:
        img_data = res.content
        file_path = os.path.join(imgdir_name, os.path.basename(img_url))
        with open(file_path, 'wb') as file:
            print(f'Writing to {file_path} ({len(img_data):,} bytes)')
            file.write(img_data)
    else:
        print(f'Error Code = {res.status_code} for {img_url}')

### 3. 파일 업로드 하기
* http://httpbin.org/post 업로드 요청을 할 수 있는 url

In [12]:
import requests

upload_files = {
    'img1' : open('f1.jpg','rb'),
    'img2': open('f2.jpg','rb'),
}
print(upload_files)

url = 'http://httpbin.org/post'
# file 업로드 하려면 requests의 post 함수에 files 속성을 사용합니다.
res = requests.post(url,files=upload_files)
print(res.status_code)
print(type(res.json()))
print(res.json()['files']['img1'])



{'img1': <_io.BufferedReader name='f1.jpg'>, 'img2': <_io.BufferedReader name='f2.jpg'>}
200
<class 'dict'>
...


### 4. 캡챠(이미지) API 호출하기
* urllib 사용
* 1. 캡차 키 발급 요청
  2. 캡차 이미지 요청
  3. 사용자 입력값 검증 요청

In [2]:
# 캡차 키 발급 요청
import os
import sys
import urllib.request
client_id = "QEtZw7_jYId8eqgUu9o4" # 개발자센터에서 발급받은 Client ID 값
client_secret = "hr2gsufDFG" # 개발자센터에서 발급받은 Client Secret 값
code = "0"
url = "https://openapi.naver.com/v1/captcha/nkey?code=" + code
#request 객체 생성
request = urllib.request.Request(url)
#요청 헤더 설정
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
#서버에 request 전달
response = urllib.request.urlopen(request)
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    print(response_body.decode('utf-8'))
else:
    print("Error Code:" + rescode)


{"key":"wOGRpixMFadWR0iq"}


In [ ]:
# 캡차 이미지 요청
import urllib.request
client_id = "QEtZw7_jYId8eqgUu9o4" # 개발자센터에서 발급받은 Client ID 값
client_secret = "hr2gsufDFG" # 개발자센터에서 발급받은 Client Secret 값
key = "wOGRpixMFadWR0iq" # 캡차 Key 값
url = "https://openapi.naver.com/v1/captcha/ncaptcha.bin?key=" + key
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
response = urllib.request.urlopen(request)
rescode = response.getcode()
if(rescode==200):
    print("캡차 이미지 저장")
    response_body = response.read()
    with open('captcha.jpg', 'wb') as f:
        f.write(response_body)
else:
    print("Error Code:" + rescode)


캡차 이미지 저장


In [ ]:
#  사용자 입력값 검증 요청
import os
from dotenv import load_dotenv
import urllib.request
client_id = "QEtZw7_jYId8eqgUu9o4" # 개발자센터에서 발급받은 Client ID 값
client_secret = "hr2gsufDFG" # 개발자센터에서 발급받은 Client Secret 값
code = "1"
key = "wOGRpixMFadWR0iq"
value = "5NYAR"
url = "https://openapi.naver.com/v1/captcha/nkey?code=" + code + "&key=" + key + "&value=" + value
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
response = urllib.request.urlopen(request)
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    print(response_body.decode('utf-8'))
else:
    print("Error Code:" + rescode)


{"result":true,"responseTime":41.5}


* requests를 사용하는 코드로 변경하기
* [requests docs](https://requests.readthedocs.io/en/latest/user/quickstart/)

In [1]:
# 사용자 입력값 검증 # 사용자 입력값 검증 요청

import os
from dotenv import load_dotenv
import requests

# .env 파일에서 환경 변수 로드
load_dotenv()

# 환경 변수에서 값 읽기
client_id = os.getenv("NAVER_CLIENT_ID")  # .env 파일의 NAVER_CLIENT_ID
client_secret = os.getenv("NAVER_CLIENT_SECRET")  # .env 파일의 NAVER_CLIENT_SECRET
key = "GNiWUAa1ksxIMhiz"
#os.getenv("NAVER_CAPTCHA_KEY")  # .env 파일의 NAVER_CAPTCHA_KEY

# 사용자 입력값
code = "1"
value = "2N53S"  # 실제 사용시 사용자 입력값으로 대체 필요

# URL 구성
# query string key1=value1&key2=value2
url = f"https://openapi.naver.com/v1/captcha/nkey?code={code}&key={key}&value={value}"

# 요청 헤더
headers = {
    "X-Naver-Client-Id": client_id,
    "X-Naver-Client-Secret": client_secret
}

# 요청 보내기
try:
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # 4xx, 5xx 오류 발생시 예외 발생
    
    # 응답 처리
    print(response.text)
    
except requests.exceptions.RequestException as e:
    print(f"Error occurred: {e}")
    print(f"Status Code: {response.status_code if 'response' in locals() else 'N/A'}")요청



SyntaxError: invalid syntax (2151899781.py, line 40)

### 5. 블로그 검색하기

In [2]:
import requests
from pprint import pprint
import os
from dotenv import load_dotenv

# .env 파일에서 환경 변수 로드
load_dotenv()

# 환경 변수에서 값 읽기
client_id = os.getenv("NAVER_CLIENT_ID")  # .env 파일의 NAVER_CLIENT_ID
client_secret = os.getenv("NAVER_CLIENT_SECRET")  # .env 파일의 NAVER_CLIENT_SECRET

headers = {
    'X-Naver-Client-Id': client_id,
    'X-Naver-Client-Secret': client_secret,
}

# query string 문자열을 dict 선언
payload = {
    'query': '파이썬',
    'display': 100,
    'sort': 'sim'
}

url = 'https://openapi.naver.com/v1/search/blog.json' #?query=파이썬&display=100&sort=sim

# requests get(url, params, headers) 요청 
res = requests.get(url, params=payload, headers=headers)
# json() 함수로 응답 결과 가져오기
items_data = res.json()['items']
print(type(res.json()))
#pprint(res.json())
print(len(items_data), type(items_data), type(items_data[0]))
pprint(items_data[0])

#[{},{},{}] 
# [] - 100 개의 데이터, {title:'',link:''} - 1개의 블로그 글 데이터

<class 'dict'>
100 <class 'list'> <class 'dict'>
{'bloggerlink': 'blog.naver.com/she_op',
 'bloggername': '오롯이 여행, 마음먹기 나름 :)',
 'description': '그래서 대전<b>파이썬</b>학원을 알아보기로 했죠. 아무 데서나 할 순 없고 커리큘럼을 단계별 실무 '
                '위주로... 기초 코딩: 프로그래밍 언어 이 과정은 대전<b>파이썬</b>학원에 처음 오는 입문자들이 '
                '프로그래밍 언어를... ',
 'link': 'https://blog.naver.com/she_op/223881011131',
 'postdate': '20250528',
 'title': '대전<b>파이썬</b>학원 단계별 코딩 실무 학습 과정'}


In [4]:
# 'title' , 'bloggername' , 'description' , 'bloggerlink' , 'link'
items_list = list()
item_list = []
for item in items_data:
#     print(item)
    item_list.append(item['title'])
    item_list.append(item['bloggername'])
    item_list.append(item['description'])
    item_list.append(item['bloggerlink'])
    item_list.append(item['link'])
    item_list.append(item['postdate'])

    items_list.append(item_list)
    item_list = []

print(items_list) #[[],[],[]]

# 'data/nhnblog.txt' 파일 생성하기
with open('data/nhnblog.txt','w',encoding="utf-8")as file:
    for items in items_list:
        for item in items:
            item = item + '\n'
            file.write(item)
        file.write('-'*150+'\n')

[['대전<b>파이썬</b>학원 단계별 코딩 실무 학습 과정', '오롯이 여행, 마음먹기 나름 :)', '그래서 대전<b>파이썬</b>학원을 알아보기로 했죠. 아무 데서나 할 순 없고 커리큘럼을 단계별 실무 위주로... 기초 코딩: 프로그래밍 언어 이 과정은 대전<b>파이썬</b>학원에 처음 오는 입문자들이 프로그래밍 언어를... ', 'blog.naver.com/she_op', 'https://blog.naver.com/she_op/223881011131', '20250528'], ['강남 AI특강, <b>파이썬</b>으로 시작하는 실전 수업', '퍼플피치', '결국 <b>파이썬</b>으로 수렴되더라고요. 문법도 어렵지 않고, 무엇보다 인공지능이나 빅데이터 같은 분야에서 거의 필수처럼 쓰인다고 하니까요. 이 강의의 커리큘럼을 살펴봤을 때 <b>파이썬</b> 중심으로 구성되어 있다는... ', 'blog.naver.com/purplepeach81', 'https://blog.naver.com/purplepeach81/223902982714', '20250618'], ['<b>파이썬</b>으로 주식 분석', '해머', '<b>파이썬</b>으로 주식 분석을 해보고 싶은 분들 제가 <b>파이썬</b>을 사용해서 주식 데이터를 수집하고 분석해본 경험을 바탕으로 정리해볼게요. 초보자도 이해할 수 있도록 전체적인 흐름부터 사용한 라이브러리... ', 'blog.naver.com/sn525v', 'https://blog.naver.com/sn525v/223924507846', '20250709'], ['대전<b>파이썬</b>학원 실전에 적용 가능한 실무 수업', '▶ 제이의 맛있는 세상 ◀', '되면서 <b>파이썬</b>과 같은 언어를 기반으로 한 학습의 필요성이 더욱 커지고 있답니다. 이러한 흐름 속에서 기초부터 실무 활용까지 단계적으로 학습할 수 있는 곳을 찾는다면 대전<b>파이썬</b>학원이 좋은 선택이 될 수... ', 'blog.naver.com/xshinerx'